# Visualizing Swin Transformer

**by Pio Lauren T. Mendoza**

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import torch 
import timm

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [3]:
print(timm.list_models("*swin*", pretrained=True))

['swin_base_patch4_window7_224', 'swin_base_patch4_window7_224_in22k', 'swin_base_patch4_window12_384', 'swin_base_patch4_window12_384_in22k', 'swin_large_patch4_window7_224', 'swin_large_patch4_window7_224_in22k', 'swin_large_patch4_window12_384', 'swin_large_patch4_window12_384_in22k', 'swin_small_patch4_window7_224', 'swin_tiny_patch4_window7_224']


In [8]:
model = timm.create_model("swin_base_patch4_window7_224")